Scrape Questions and metadata

In [1]:
from leetscrape.GetQuestionsList import GetQuestionsList
from leetscrape.GetQuestionInfo import GetQuestionInfo
from leetscrape.utils import (
    combine_list_and_info,
    get_all_questions_body,
    extract_solutions,
    upload_solutions,
)
import pandas as pd
import pickle
from sqlalchemy import create_engine, update
from dotenv import dotenv_values



Query the question list

In [4]:
ls = GetQuestionsList()
ls.scrape() # Scrape the list of questions
ls.to_csv(directory_path="../data_test/") # Save the scraped tables to a directory

Note: The default ALL_JSON_URL might be out-of-date. Please update it by going to https://leetcode.com/problemset/all/ and exploring the Networks tab for a query returning all.json.
Scraping companies ... Done
Scraping questions list ... Done
Extracting question topics ... Done
Getting Categories ... Done
Scraping Topic Tags ... Done
Extracting question category ... Done


Query Individual question info such as the body, test cases, constraints, hints, code stubs, etc.

In [2]:
# # This table can be generated using the previous commnd using
# # questions_info = ls.questions
# questions_info = pd.read_csv("../data/questions.csv")

# # Scrape question body
# questions_body_list = get_all_questions_body(
#     questions_info["titleSlug"].tolist(),
#     questions_info["paidOnly"].tolist(),
#     save_to="../data/questionBody.pickle",
# )

# # Save to a pandas dataframe
# questions_body = pd.DataFrame(
#     questions_body_list
# ).drop(columns=["titleSlug"])
# questions_body["QID"] = questions_body["QID"].astype(int)


""" Run the above code stub once and save the data as a pickle file. Using this data
from now on since the above code stub is time consuming. """

with open("../data/questionBody.pickle", "rb") as f:
    data = pickle.load(f)
questions_body = pd.DataFrame(data).drop(columns=["titleSlug"])
questions_body["QID"] = questions_body["QID"].astype(int)


Create a new dataframe with all the questions with their metadata and body information.

In [6]:
questions = combine_list_and_info(info_df = questions_body, list_df=ls.questions, save_to="../data/all.json")

Data Upload to SUPABASE table|s

In [3]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from dotenv import dotenv_values

config = dotenv_values("../.env")


In [22]:
engine = create_engine(
    f"postgresql://{config['SUPABASE_USERNAME']}:{config['SUPABASE_PASSWORD']}@{config['SUPABASE_HOSTNAME']}:{config['SUPABASE_PORT']}/{config['SUPABASE_DBNAME']}",
    echo=True,
)
questions.to_sql(con=engine, name="questions", if_exists="append", index=False)
ls.topicTags.to_sql(con=engine, name="topic_tags", if_exists="append", index=False)
ls.categories.to_sql(con=engine, name="categories", if_exists="append", index=False)
ls.companies.to_sql(con=engine, name="companies", if_exists="append", index=False)
ls.questionTopics.to_sql(
    con=engine, name="question_topics", if_exists="append", index=True, index_label="id"
)
ls.questionCategory.to_sql(
    con=engine,
    name="question_category",
    if_exists="append",
    index=True,
    index_label="id",
)


Upload solutions to questions

In [2]:
config = dotenv_values("../../.env")
eng = create_engine(
    f"postgresql://{config['SUPABASE_USERNAME']}:{config['SUPABASE_PASSWORD']}@{config['SUPABASE_HOSTNAME']}:{config['SUPABASE_PORT']}/{config['SUPABASE_DBNAME']}",
    echo=False,
)

In [3]:
solutions = extract_solutions("../solutions/q_0001_TwoSum.py")
upload_solutions(eng, 1, solutions[1])